In [4]:
import pandas as pd
import sklearn
from sklearn.cross_validation import KFold
import math as m
import numpy as np
import decimal as dec
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

# dec.getcontext().prec = 5
        
def classify(d,Fun):
    c1=pd.DataFrame(columns=('c1','ALabel', 'PLabel','FunLabel'))
    c2=pd.DataFrame(columns=('c2', 'ALabel','PLabel','FunLabel'))
    for i in range(len(d)):
        if(d.d[i]>0):
            c1.loc[i]=[d.d[i],d.Label[i],1,Fun[i]]
        else:
            c2.loc[i]=[d.d[i],d.Label[i],0,Fun[i]]
#     print c1
#     print c2
    c=pd.concat([c1,c2])
    return c
        
def calculate_parameters(x,c1,c2):
    alpha1=[]
    alpha2=[]
    for i in range(0,54):
        a1=float(sum(x[i][0:len(c1)]))/float(len(c1))
        alpha1.append(a1)
        a2=float(sum(x[i][len(c1):len(x)]))/float(len(c2))
        alpha2.append(a2)
    spam_prior= float(len(c1))/float(len(x))
    nospam_prior=float(len(c2))/float(len(x))
    return alpha1, alpha2, spam_prior,nospam_prior

def member(alpha1, alpha2, spam_prior,nospam_prior,x,y):
    g1=[]
    g2=[]
    val1=[]
    val2=[]
    val11=[]
    val22=[]
    d=pd.DataFrame(columns=('d', 'Label'))
    for i in alpha1:
        if i != 0.0 and i != 1.0:
            val1.append(m.log(i))
            val2.append(m.log(1 - i))
        else:
            val1.append(0.0)
            val2.append(0.0)
    for i in alpha2:
        if i != 0.0 and i != 1.0:
            val11.append(m.log(i))
            val22.append(m.log(1 - i))
        else:
            val11.append(0.0)
            val22.append(0.0)
    g1 = [(np.sum(((x[j][i]*val1[j])+((x[j][i])*val2[j])) for j in range(0,54)) + m.log(spam_prior)) \
          for i in range(len(x))]
    g2 = [(np.sum(((x[j][i]*val11[j])+((x[j][i])*val22[j])) for j in range(0,54)) + m.log(nospam_prior)) \
          for i in range(len(x))]
    for i in range(len(x)):
        d.loc[i]=[g1[i]-g2[i], y[i]]
    return d

def confusion_mat(c):
    TP=0
    FP=0
    TN=0
    FN=0
    for i in range(len(c)):
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==1):
            TP+=1
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==0):
            TN+=1
        if(c.ALabel[i]==0 and c.PLabel[i]==1):
            FP+=1
        if(c.ALabel[i]==1 and c.PLabel[i]==0):
            FN+=1
    con_mat=np.matrix([[TP,FP],[FN,TN]])
    p1= (float(TP)/float((TP+FP)))
    r1=(float(TP)/float((TP+FN)))
    p2= (float(TN)/float((TN+FN)))
    r2=(float(TN)/float(TN+FP))
    Accuracy=float((TP+TN))/float((TP+FP+TN+FN))
    F1=(2*(float((p1*r1))/float((p1+r1))))
    F2=(2*(float((p2*r2))/float((p2+r2))))
    precision=[p1,p2]
    recall=[r1,r2]
    print "Confusion Matrix: ",con_mat
    print "Accuracy: ",Accuracy
    print "Precision: ",precision
    print "Recall: " ,recall
    print "F_measure: ",[F1,F2]
    
        
def main():
    filename='spambase.txt'
    rows=[]
    y=[]
    c1=[]
    c2=[]    
    x=pd.DataFrame(columns=range(54))
    lines = [line.rstrip('\n') for line in open(filename)]
    for i in range(len(lines)):
        row=lines[i].split(',')
        rows.append(row)
        y1=int(rows[i][57])
        y.append(y1)
        x.loc[i]=rows[i][0:54]
        for j in range(len(rows[i])-4):
            x[j][i]= float(x[j][i])
            if(x[j][i]>0):
                x[j][i]=1
            else:
                x[j][i]=0
    for i in range(len(y)):
        if(y[i]==1):
            c=y[i]
            c1.append(c)
        else:
            c=y[i]
            c2.append(c)
    X=np.matrix(x)
#     without fold
    print("Performance with 75% training size")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    alpha1, alpha2, spam_prior,nospam_prior=calculate_parameters(X_train,c1,c2)
    d=member(alpha1, alpha2, spam_prior,nospam_prior,X_test,y_test)
    clf = BernoulliNB()
    clf.fit(np.matrix(X_train), y_train)
    Fun1=clf.predict(np.matrix(X_test))
    c=classify(d,Fun1)
    confusion_mat(c)

    
if __name__=='__main__':
    main()

Performance with 75% training size
[[570 744]
 [  7  60]]
Accuracy:  0.456191165822
Precision:  [0.4337899543378995, 0.8955223880597015]
Recall:  [0.9878682842287695, 0.07462686567164178]
F_measure:  [0.6028556319407721, 0.1377726750861079]
